# Prototipo per un software gestionale destinato a un negozio di prodotti vegani

L'obiettivo di questo notebook è mostrare un prototipo per un software gestionale destinato a un negozio di prodotti vegani.

Nelle specifiche si chiedeva che il software disponesse delle seguenti funzionalità:
* Registrare l'arrivo di nuovi prodotti in magazzino;
* Registrare le vendite effettuate;
* Mostrare a video l'elenco dei prodotti disponibili in magazzino:
* Calcolare e stampare il profitto lordo, ossia la somma dei guadagni ottenuti dalle vendite;
* Calcolare e stampare il profitto netto, ossia la differenza tra il profitto lordo e la somma degli acquisti;
* Stampare un menu dei comandi disponibili.

Sempre secondo le specifiche, il programma doveva possedere le seguenti caratteristiche:
* Essere permanente, quindi i dati inseriti dovevano esistere al di fuori del programma stesso memorizzati su file;
* I dati inseriti devono essere validati e corretti;
* Il programma è testuale, cioè i comandi dovevano essere inseriti tramite tastiera dall'utente.

Ogni sezione del notebook mostrerà come è stato svolto il progetto. In particolare, dopo aver mostrato il menu di aiuto nella prima sezione, sarà spiegato come il programma gestisce i flussi di vendita e di acquisto. Dopo il calcolo dei profitti lordi e netti, il notebook si conclude con il prototipo del programma e un esempio del suo funzionamento.

## Il menu di aiuto

Per realizzare il menu di aiuto, è stato prima definito il dizionario *help_dict*, che a ogni comando disponibile associa una breve descrizione del comando stesso.

In [1]:
help_dict={"aiuto" : "Spiega i comandi che puoi usare",
           "registra":"Registra i nuovi arrivi in magazzino",
           "vendi":"Registra le nuove vendite",
           "magazzino":"Riepiloga le disponibilità di magazzino",
           "lordo":"Mostra i profitti lordi",
           "netto":"Mostra i profitti netti",
           "esci":"Chiude il programma"}

La funzione **ask_help** ha il compito sia di mostrare il menu che di validare il comando inserito. L'idea è che, nel caso in cui l'utente sbagli a inserire il comando, il menu di aiuto venga comunque stampato per guidarlo nell'uso del programma.

In [2]:
def ask_help(command="aiuto"):
  if command not in help_dict.keys():
    print("Comando non riconosciuto")

  print("Questi sono i comandi che puoi utilizzare")
  for key in help_dict.keys():
    print(f"{key} = {help_dict[key]}")

## Flussi di magazzino

In questa sezione sarà mostrato come il programma gestisce i flussi di magazzino. Dopo aver illustrato l'oggetto Product e le funzioni di supporto, le ultime tre sottosezioni saranno dedicate agli ingressi, alle uscite e ai riepiloghi di magazzino.

### L'oggetto Product

Dal momento che lo scopo del programma è la gestione di un magazzino di prodotti, si è deciso di creare un apposito oggetto chiamato appunto *Product* che può essere manipolato dal programma a partire dagli attributi di seguito elencati:
* Una stringa *name* con il nome del prodotto convertito in minuscolo;
* Un intero *quantity* che indica la quantità di prodotto;
* I decimali *sell_price* e *buy_price* che indicano i prezzi di vendita e di acquisto rispettivamente.

A questi attributi sono stati aggiunti i metodi:
* **sell_total** e **buy_total** per il calcolo del totale di vendita e di acquisto;
* **create_row**, che crea la riga di dati per l'inserimento nel file CSV;
* **print_selling** e **print_buying** che stampano il totale di vendita e di acquisto;
* Il metodo speciale per la stampa dell'oggetto.

L'utilizzo di questi metodi sarà approfondito nelle prossime sottosezioni.

In [3]:
class Product:
  def __init__(self, name:str, quantity:int, sell_price:float, buy_price:float):
    self.name = name
    self.quantity = quantity
    self.sell_price = sell_price
    self.buy_price = buy_price

  def sell_total(self):
    return self.quantity * self.sell_price

  def buy_total(self):
    return self.quantity * self.buy_price

  def create_row(self):
    return [self.name, self.quantity, self.sell_price, self.buy_price]

  def print_buying(self):
    print(f"{self.name} X {self.quantity} = € {self.buy_total()}")

  def print_selling(self):
    print(f"{self.name} X {self.quantity} = € {self.sell_total()}")

  def __repr__(self):
    return f"{self.name}\t{self.quantity}\t{self.sell_price}\t{self.buy_price}\n"

### Funzioni di supporto

Le funzioni di supporto in questa sezione hanno lo scopo di sostenere il lavoro delle funzioni principali nel programma.

La prima funzione, **validate_integer**, ha lo scopo di validare le quantità di prodotto inserite da tastiera. Partendo dal presupposto che i prodotti siano disponibili in confezioni, non ha senso parlare di quantità decimali di confezioni per cui le quantità inserite devono essere intere.

In [4]:
def validate_integer(quantity):
  try:
    quantity_int = int(quantity)
    return quantity_int, True
  except ValueError:
    print("La quantità inserita deve essere un intero, riprova.")
    return 0, False

In modo analogo, **validate_price** fa in modo che i prezzi inseriti da tastiera siano numeri decimali e positivi.

In [5]:
def validate_price(price):
  try:
    price_float = float(price)
    assert price_float>0, "Il prezzo deve essere un numero positivo"
    return price_float, True
  except AssertionError:
    print("Il prezzo deve essere un numero positivo, riprova.")
    return 0.0, False
  except ValueError:
    print("Il prezzo deve essere un numero con la virgola, riprova.")
    return 0.0, False

Infine, **validate_answer** valida le risposte alle domande che richiedono sì/no come risposta.

In [8]:
def validate_answer(answer):
  try:
    assert answer in ("y","n"), "La risposta inserita deve essere y/n."
    return answer, True
  except AssertionError:
    print("La risposta inserita non è valida, riprova.")
    return "", False

Infine, **create_file** crea i file CSV per gli ingressi, le uscite e i riepiloghi di magazzino qualora questi non esistano (come, ad esempio, può accadere in un caso di primo utilizzo).

In [11]:
import csv
import os

entries_csv="entries.csv"
sales_csv="sales.csv"
store_csv="store.csv"

def create_file(name_path):
  if not os.path.exists(name_path):
    file=open(name_path, "w")
    file.close()
    print(f"Il file {name_path} è stato creato")

### Registrare gli acquisti

La registrazione degli acquisti nella funzione **entries** si avvale dei seguenti passi:
1. Dopo l'inserimento del nome e della quantità, il programma controlla se il prodotto è presente in magazzino e, in caso positivo, recupera le informazioni sui prezzi, altrimenti li chiede da tastiera;
2. Il programma inserisce il prodotto alla fine del file e ne stampa un riepilogo tramite il metodo **print_buying**;
3. Il programma chiede se si vuole inserire un altro prodotto e, in caso affermativo, ripete dal passo 1.

In [12]:
def entries():
  answer = ""
  while answer.lower() != "n":
    name = input("Inserire il nome del prodotto: ").lower()

    flag = False
    while not flag:
      quantity, flag=validate_integer(input("Inserire una quantità: "))

    store_list=store(to_print=False)

    if name not in [x.name for x in store_list]:
      flag = False
      while not flag:
        sell_price, flag=validate_price(input("Inserire il prezzo di vendita: "))

      flag = False
      while not flag:
        buy_price, flag=validate_price(input("Inserire il prezzo di acquisto: "))
    else:
      id_item=[x.name for x in store_list].index(name)
      sell_price=store_list[id_item].sell_price
      buy_price=store_list[id_item].buy_price

    item=Product(name, quantity, sell_price, buy_price)

    with open(entries_csv, 'a', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(item.create_row())

    item.print_buying()

    flag = False
    while not flag:
        answer, flag=validate_answer(input("Vuoi inserire un altro prodotto? (y/n) ").lower())

### Registrare le vendite

In modo simile agli acquisti, la registrazione delle vendite avviene tramite i seguenti step:
1. Dopo aver inserito il nome e la quantità di prodotto, il programma controlla se il prodotto è disponibile in magazzino e nella giusta quantità e, in caso affermativo, recupera le informazioni sui prezzi;
2. La vendita viene inserita alla fine del file e ne viene stampato un riepilogo tramite il metodo **print_selling**;
3. Il programma chiede se si vuole aggiungere un'altra vendita e, in caso affermativo, ripete dal passo 1.

In [13]:
def sales():
  answer = ""
  while answer.lower() != "n":
    name = input("Inserire il nome del prodotto: ")
    flag = False
    while not flag:
      quantity, flag=validate_integer(input("Inserire la quantità: "))

    store_list=store(to_print=False)
    if name not in [x.name for x in store_list]:
      print("Il prodotto non è presente in magazzino")
    elif store_list[0].quantity<quantity:
      print("Non c'è disponibilità in magazzino")
    else:
      id_item=[x.name for x in store_list].index(name)
      sell_price=store_list[id_item].sell_price
      buy_price=store_list[id_item].buy_price

      item=Product(name, quantity, sell_price, buy_price)

      with open(sales_csv, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(item.create_row())

      item.print_selling()

    flag = False
    while not flag:
        answer, flag=validate_answer(input("Vuoi inserire un altro prodotto? (y/n) ").lower())

### Riepilogo di magazzino

La funzione **store** si basa sullo stesso principio alla base del calcolo dei saldi dei movimenti bancari. Nello specifico, questa funzione prende in ingresso il parametro booleano *to_print* a cui è assegnato di default il valore True.

Premesso questo, la funzione si articola nei seguenti passi:
1. Il programma memorizza i dati di entrata e di uscita dal magazzino nella lista *flows*;
2. Il programma crea la lista di oggetti Product *store_list* dove, una volta estratti in modo univoco il nome e i prezzi dei prodotti, la quantità viene calcolata come differenza tra la somma degli acquisti e la somma delle vendite;
3. Gli elementi di *store_list* vengono salvati in un file CSV;
4. Se *to_print* è True, il riepilogo di magazzino viene stampato;
5. Il programma ritorna la lista *store_list*.

La funzione ha così il duplice scopo di fare da supporto alle registrazioni di vendita e di acquisto e, al contempo, soddisfare una delle funzionalità richieste per il programma.

In [14]:
def store(to_print=True):
  flows=[]
  with open(entries_csv, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      flows.append(Product(row[0], int(row[1]), float(row[2]), float(row[3])))

  with open(sales_csv, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      flows.append(Product(row[0], -int(row[1]), float(row[2]), float(row[3])))

  store_list=[]
  for item in flows:
    if item.name not in [x.name for x in store_list]:
      name=item.name
      sell_price=item.sell_price
      buy_price=item.buy_price
      quantity=0
      for x in flows:
        if x.name==name:
          quantity+=x.quantity
      store_list.append(Product(name, quantity, sell_price, buy_price))

  with open(store_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    for item in store_list:
      writer.writerow(item.create_row())

  if to_print:
    print(f"name \t quantity \t sell_price \t buy_price")
    for item in store_list:
      print(item)

  return store_list

## Calcolo dei profitti

Utilizzando come supporto la funzione **store**, la funzione **gros_profit** calcola il profitto lordo del magazzino.

In [15]:
def gros_profit():
  store_list=store(to_print=False)
  gross_profit=0
  for item in store_list:
    gross_profit+=item.sell_total()

  print(f"Il profitto lordo è di € {gross_profit}")

In modo analogo, la funzione **net_profit** calcola il profitto netto come somma delle differenze tra i totali di vendita e di acquisto.

In [16]:
def net_profit():
  store_list=store(to_print=False)
  net_profit=0
  for item in store_list:
    net_profit+=item.sell_total()-item.buy_total()

  print(f"Il profitto netto è di € {net_profit}")

## Il prototipo finale

Le funzioni mostrate nelle sezioni precedenti sono state riunite nella funzione **main**, che agisce come segue:
1. Controlla se esistono i file di supporto al programma;
2. Prende in ingresso il comando da tastiera e chiama la funzione corrispondente;
3. In caso di uscita, il programma risponde con un saluto e chiude.

In [17]:
def main():
  create_file(entries_csv)
  create_file(sales_csv)
  create_file(store_csv)

  command = ""
  while command.lower() != "esci":
    command = input("Inserisci un comando: ").lower()
    if not command in help_dict.keys():
      ask_help(command=command)
    elif command == "aiuto":
      ask_help()
    elif command == "registra":
      entries()
    elif command == "vendi":
      sales()
    elif command == "magazzino":
      store()
    elif command == "lordo":
      gros_profit()
    elif command == "netto":
      net_profit()
    else:
      print("Arrivederci!")
      exit()

Si mostra di seguito un caso di primo utilizzo del programma.

In [18]:
main()

Il file entries.csv è stato creato
Il file sales.csv è stato creato
Il file store.csv è stato creato
Inserisci un comando: aiuto
Questi sono i comandi che puoi utilizzare
aiuto = Spiega i comandi che puoi usare
registra = Registra i nuovi arrivi in magazzino
vendi = Registra le nuove vendite
magazzino = Riepiloga le disponibilità di magazzino
lordo = Mostra i profitti lordi
netto = Mostra i profitti netti
esci = Chiude il programma
Inserisci un comando: registra
Inserire il nome del prodotto: latte di soia
Inserire una quantità: 10
Inserire il prezzo di vendita: 2.99
Inserire il prezzo di acquisto: 1.99
latte di soia X 10 = € 19.9
Vuoi inserire un altro prodotto? (y/n) y
Inserire il nome del prodotto: burger di tofu
Inserire una quantità: 20
Inserire il prezzo di vendita: 4.49
Inserire il prezzo di acquisto: 2.99
burger di tofu X 20 = € 59.800000000000004
Vuoi inserire un altro prodotto? (y/n) no
La risposta inserita non è valida, riprova.
Vuoi inserire un altro prodotto? (y/n) n
Inser

## Conclusione

In questo notebook è stato mostrato lo sviluppo di un software gestionale per il magazzino di un negozio di prodotti vegani a partire da determinati requisiti.

Nella prima sezione è stato mostrato il menu di aiuto per guidare l'utente nell'utilizzo del programma e validare i comandi inseriti da tastiera.

Nella seconda sezione, dopo aver mostrato l'oggetto Product e alcune funzioni di supporto al programma, è stata illustrata la gestione degli ingressi (acquisti), delle uscite (vendite) e dei riepiloghi di magazzino.

Nella terza parte, utilizzando i metodi definiti in Product, è stato mostrato il calcolo dei profitti lordi e netti.

Nell'ultima sezione, invece, le funzioni viste sono state incorporate nella funzione **main** ed è stato mostrato un caso di utilizzo.